In [2]:
def calAct(self, q_in, actmethod):
    q_in = np.array(q_in)
    if actmethod == 'tanh':
        q_out = 2/(1+np.exp(-2*q_in))-1
    elif actmethod == 'relu':
        q_out = q_in*(q_in>0)
    elif actmethod == 'sigmoid':
        q_out = 1/(1+np.exp(-q_in))


    return np.matrix(q_out)

# takes in power system data, removes classical parameters (param_compact) and finds optimal quantum parameters to input into the quantum circuit
def param_compact(self, param, x_sample): 
    n_qubits = self.n_qubits
    n_layers = self.n_layers
    nQAOA = self.nQAOA
    n_x = self.n_x
    """translate param_unaug from param
    param_compact: n_layers * (2*n_qubits + 2*n_qubits)
    param_qnn: n_layers * (2*n_qubits + 2*n_qubits*(n_x+1) )
    """
    param_m = np.matrix(param)
    x_aug = np.matrix(np.concatenate((x_sample,[1]))).T

    param_compact_ = np.matrix( np.zeros(( n_layers , nQAOA + 2*n_qubits )) )
    param_compact_[:,:nQAOA] = param_m[:,:nQAOA]

    iindx = nQAOA
    iindx_compact = nQAOA
    for i_qubit in range(2*n_qubits): # only use n_qubits not x2 because we only do encoding once
        # in every layer, we have 2*n_qubit block for embedding x by activation function
        # need to embed x to params for pennylane
        # theoretically, 2*nq blocks are identical
        q_in = param_m[:,iindx:(iindx+n_x+1)]*x_aug
        q_out = self.calAct(q_in, self.activation)
        param_compact_[:,iindx_compact] = q_out
        iindx = iindx + n_x+1
        iindx_compact = iindx_compact+1


    # x_con = [0]
    return np.array(param_compact_)

def grad_augment(self, grad_compact, param , x_sample):
    """
    """
    n_qubits = self.n_qubits
    n_layers = self.n_layers
    nQAOA = self.nQAOA
    n_x = self.n_x
    """translate grad from grad_aug
    grad: n_layers * (2*n_qubits + 2*n_qubits)
    grad_aug: n_layers * (2*n_qubits + 2*n_qubits*(n_x+1) )
    """
    #   grad_aug
    param_m = np.matrix(param)
    grad_compact = np.matrix(grad_compact)
    x_aug = np.matrix(np.concatenate((x_sample,[1]))).T

    grad_aug = np.matrix(np.zeros(param.shape))
    grad_aug[:,:nQAOA] = grad_compact[:,:nQAOA]

    iindx = nQAOA
    iindx_compact = nQAOA
    for i_qubit in range(2*n_qubits):
        # in every layer, we have 2*n_qubit block for embedding x by activation function
        # need to recalculate gradient for them
        # theoretically, 2*nq blocks are identical

        q_in = param_m[:,iindx:(iindx+n_x+1)]*x_aug
        q_out = self.calAct_dev(q_in, self.activation)
        grad_qout = np.multiply(grad_compact[:,iindx_compact],q_out)
        grad_aug[:,iindx:(iindx+n_x+1)] = grad_qout*x_aug.T
        # param_m_line = param_m[:,iindx:(iindx+n_x+1)].reshape(1,-1)[0][0]
        # grad_aug_server[:,iindx:(iindx+n_x+1)] = grad_qout*param_m_line[0,iindx_server:iindx_server+n_x+1]

        iindx = iindx + n_x+1
        iindx_compact = iindx_compact+1


    return np.array(grad_aug)